# Cleaning Data

In [26]:
import pandas as pd
import numpy as np
from pandas import *
from numpy import *

We merge the oil price dataset from the Federal Research Economic Data (oil_price.csv) and the exchange rate data set from the Bank of Canada (LEGACY_NOON_RATES.csv). 

In [27]:
oil_price = pd.read_csv("oil_price.csv")
ts_lst = ["date","IEXE0701","IEXE1201", "IEXE0101", "EUROCAE01"]

rate = pd.read_csv("LEGACY_NOON_RATES.csv",skiprows=list(range(0,81))+list(range(2692,2814)))[ts_lst]
data = pd.merge(oil_price, rate, how='inner', left_on = "DATE", right_on="date")
rename_dict = {"DCOILWTICO": "Oil_Price", 
               "IEXE0701": "CAD_JPY", 
               "IEXE1201":"CAD_GBP", 
               "IEXE0101":"CAD_USD", 
               "EUROCAE01":"CAD_EUR"}
data.rename(index=str, columns=rename_dict, inplace = True)
data.drop("date",axis=1, inplace = True)

data = data.replace('.', np.nan)
data.dropna(axis=0, how='any', inplace = True)


In [28]:
data.head()

,DATE,Oil_Price,CAD_JPY,CAD_GBP,CAD_USD,CAD_EUR
0,2007-05-01,64.43,0.009257,2.2171,1.1089,1.5081
1,2007-05-02,63.78,0.009235,2.2073,1.1087,1.5074
2,2007-05-03,63.23,0.009199,2.2005,1.1070,1.5017
3,2007-05-04,61.89,0.009211,2.2062,1.1069,1.5040
4,2007-05-07,61.48,0.009189,2.1986,1.1024,1.5010


## Remove any row with Missing Data

There are various ways to handling missing data. For example, we can pad missing values with their previous values, assign an unique value or remove the row.

In our case, every missing entry corresponds to holidays when markets are closed. Therefore, we choose to remove these rows.

In [29]:
data = data.replace('.', np.nan)
data.dropna(axis=0, how='any', inplace = True)

## Generate Lagged Variables

When we forecast a daily noon exchange rate, we do not know other daily noon exchange rates and other daily oil prices. The latest piece of information available is yesterday's noon exchange rates. Therefore, we generate several lagged variables for prediction.

The number of lags means how many days of information from the past we use to forecast today's exchange rate.  

In [30]:
var_lst = data.columns.tolist()
num_lags = 7
for var in var_lst[1:]:
    for lag in range(1,num_lags + 1):
        col_name = "L"+str(lag)+"."+str(var)
        data[col_name] = data[var].shift(-lag)

Generating lagged variables produces missing values. Similar to the above, we remove rows with missing values.

In [31]:
data.dropna(axis=0, how='any', inplace = True)

In [32]:
data.head()

,DATE,Oil_Price,CAD_JPY,CAD_GBP,CAD_USD,CAD_EUR,L1.Oil_Price,L2.Oil_Price,L3.Oil_Price,L4.Oil_Price,...,L5.CAD_USD,L6.CAD_USD,L7.CAD_USD,L1.CAD_EUR,L2.CAD_EUR,L3.CAD_EUR,L4.CAD_EUR,L5.CAD_EUR,L6.CAD_EUR,L7.CAD_EUR
0,2007-05-01,64.43,0.009257,2.2171,1.1089,1.5081,63.78,63.23,61.89,61.48,...,1.1048,1.1064,1.1073,1.5074,1.5017,1.5040,1.5010,1.4950,1.4991,1.4962
1,2007-05-02,63.78,0.009235,2.2073,1.1087,1.5074,63.23,61.89,61.48,62.26,...,1.1064,1.1073,1.1135,1.5017,1.5040,1.5010,1.4950,1.4991,1.4962,1.5053
2,2007-05-03,63.23,0.009199,2.2005,1.1070,1.5017,61.89,61.48,62.26,61.54,...,1.1073,1.1135,1.1069,1.5040,1.5010,1.4950,1.4991,1.4962,1.5053,1.4991
3,2007-05-04,61.89,0.009211,2.2062,1.1069,1.5040,61.48,62.26,61.54,61.85,...,1.1135,1.1069,1.0977,1.5010,1.4950,1.4991,1.4962,1.5053,1.4991,1.4932
4,2007-05-07,61.48,0.009189,2.1986,1.1024,1.5010,62.26,61.54,61.85,62.35,...,1.1069,1.0977,1.1029,1.4950,1.4991,1.4962,1.5053,1.4991,1.4932,1.4912


## Make Date a Timestamp

In [33]:
type(data["DATE"][0])

str

In [34]:
data["DATE"] = data["DATE"].map(lambda x: datetime.strptime(x, "%Y-%m-%d"))

In [35]:
type(data["DATE"][0])

pandas._libs.tslib.Timestamp

## Set DATE as the index

In [36]:
data.set_index("DATE", inplace = True)

## Convert Every Cell from String to Float

In [37]:
var_lst = data.columns.tolist()
for var in var_lst:
    data[var] = data[var].astype(float)

In [38]:
data.head()

,Oil_Price,CAD_JPY,CAD_GBP,CAD_USD,CAD_EUR,L1.Oil_Price,L2.Oil_Price,L3.Oil_Price,L4.Oil_Price,L5.Oil_Price,...,L5.CAD_USD,L6.CAD_USD,L7.CAD_USD,L1.CAD_EUR,L2.CAD_EUR,L3.CAD_EUR,L4.CAD_EUR,L5.CAD_EUR,L6.CAD_EUR,L7.CAD_EUR
DATE,,,,,,,,,,,,,,,,,,,,,
2007-05-01,64.43,0.009257,2.2171,1.1089,1.5081,63.78,63.23,61.89,61.48,62.26,...,1.1048,1.1064,1.1073,1.5074,1.5017,1.5040,1.5010,1.4950,1.4991,1.4962
2007-05-02,63.78,0.009235,2.2073,1.1087,1.5074,63.23,61.89,61.48,62.26,61.54,...,1.1064,1.1073,1.1135,1.5017,1.5040,1.5010,1.4950,1.4991,1.4962,1.5053
2007-05-03,63.23,0.009199,2.2005,1.1070,1.5017,61.89,61.48,62.26,61.54,61.85,...,1.1073,1.1135,1.1069,1.5040,1.5010,1.4950,1.4991,1.4962,1.5053,1.4991
2007-05-04,61.89,0.009211,2.2062,1.1069,1.5040,61.48,62.26,61.54,61.85,62.35,...,1.1135,1.1069,1.0977,1.5010,1.4950,1.4991,1.4962,1.5053,1.4991,1.4932
2007-05-07,61.48,0.009189,2.1986,1.1024,1.5010,62.26,61.54,61.85,62.35,62.55,...,1.1069,1.0977,1.1029,1.4950,1.4991,1.4962,1.5053,1.4991,1.4932,1.4912
